In [1]:
import glob
import pandas as pd
cat = glob.glob('train/cat.*.jpg')
dog = glob.glob('train/dog.*.jpg')
data = pd.DataFrame({
    'path':cat+dog,
    'ans':[0]*len(cat) + [1]*len(dog)
})
data

,path,ans
0,train\cat.0.jpg,0
1,train\cat.1.jpg,0
2,train\cat.10.jpg,0
3,train\cat.100.jpg,0
4,train\cat.1000.jpg,0
...,...,...
24995,train\dog.9995.jpg,1
24996,train\dog.9996.jpg,1
24997,train\dog.9997.jpg,1
24998,train\dog.9998.jpg,1


In [2]:
# include_top= whether should the cnn do MLP?answer is no here
# input_shape=224*224 as the same for imageNet comp. for the best performance
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224,224,3))
vgg.summary()

Using TensorFlow backend.
W1007 15:51:26.892546 16992 deprecation_wrapper.py:119] From C:\Users\Big data\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1007 15:51:26.906529 16992 deprecation_wrapper.py:119] From C:\Users\Big data\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1007 15:51:26.908505 16992 deprecation_wrapper.py:119] From C:\Users\Big data\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1007 15:51:26.927479 16992 deprecation_wrapper.py:119] From C:\Users\Big data\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4267: Th

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
def test():
    return print
test()('hello')

hello


In [4]:
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
# Dense(256, acitivation='relu') -> function(e.g. return print)
# tensor = func(tensor)
for l in vgg.layers:
    l.trainable = False
x = Flatten()(vgg.output)
x = Dense(2048,activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(2, activation='softmax')(x)
# Model(inputs = tensor, outputs = tensor)
cnn = Model(inputs=vgg.input, outputs=x)
cnn.summary()

W1007 15:51:27.597232 16992 deprecation.py:506] From C:\Users\Big data\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [5]:
# trainable should be set b4 compiling
cnn.compile(loss='categorical_crossentropy',
           optimizer='adam',
           metrics=['accuracy'])

W1007 15:51:27.707901 16992 deprecation_wrapper.py:119] From C:\Users\Big data\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [6]:
from sklearn.model_selection import train_test_split
# !do not use list or dataframe after training phase, use np.array ONLY
import numpy as np
from keras.utils.np_utils import to_categorical
x = np.array(data['path'])
y = np.array(data['ans'])
y_cat = to_categorical(y)
x_train, x_test, y_train, y_test = train_test_split(x,y_cat,test_size=0.1)

In [7]:
# pillow(pil)
from keras.preprocessing.image import load_img
# preprocessing depends on the model used
from keras.applications.vgg16 import preprocess_input
img = load_img(x_train[0], target_size=(224,224)).convert('RGB')
img_np = np.array(img)
preprocess_input(img_np)

array([[[120.061    , 110.221    , 101.32     ],
        [120.061    , 110.221    , 101.32     ],
        [120.061    , 110.221    , 101.32     ],
        ...,
        [-26.939003 , -29.779    , -36.68     ],
        [-14.939003 , -17.779    , -24.68     ],
        [ -3.939003 ,  -5.7789993, -14.68     ]],

       [[120.061    , 110.221    , 101.32     ],
        [120.061    , 110.221    , 101.32     ],
        [120.061    , 110.221    , 101.32     ],
        ...,
        [-74.939    , -77.779    , -84.68     ],
        [-78.939    , -81.779    , -88.68     ],
        [-79.939    , -82.779    , -89.68     ]],

       [[120.061    , 110.221    , 101.32     ],
        [120.061    , 110.221    , 101.32     ],
        [120.061    , 110.221    , 101.32     ],
        ...,
        [ 13.060997 ,  10.221001 ,   3.3199997],
        [ 11.060997 ,   8.221001 ,   1.3199997],
        [  6.060997 ,   1.2210007,  -5.6800003]],

       ...,

       [[108.061    ,  99.221    ,  87.32     ],
        [10

In [8]:
# x=pic, y=ans, batch=qty
# return(original pics)
def get_data(x,y,batch):
    idx = np.random.randint(0,x.shape[0],batch)
    oriimgs = []
    preimgs = []
    for p in x[idx]:
        img = load_img(p, target_size=(224,224)).convert('RGB')
        img_np = np.array(img)
        oriimgs.append(img_np)
        img_pre = preprocess_input(img_np)
        preimgs.append(img_pre)
    oriimgs = np.array(oriimgs)
    preimgs = np.array(preimgs)
    ans = y[idx]
    return (oriimgs, preimgs, ans)

In [9]:
# train_on_batch
# test_on_batch
for i in range(10):
    print('times',i+1)
    batch = 10
    train = get_data(x_train, y_train, batch)
    train_result = cnn.train_on_batch(train[1],train[2])
    print('train=',train_result)
    # more validation pics, higher accuracy
    test_batch = 20
    test = get_data(x_test,y_test,test_batch)
    test_result = cnn.test_on_batch(test[1],test[2])
    print('test=',test_result)
    print('-'*30)

W1007 15:51:28.672694 16992 deprecation.py:323] From C:\Users\Big data\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


times 1
train= [7.7213697, 0.3]
test= [1.3698113, 0.9]
------------------------------
times 2
train= [2.9400983, 0.8]
test= [7.2531433, 0.55]
------------------------------
times 3
train= [9.670857, 0.4]
test= [10.476762, 0.35]
------------------------------
times 4
train= [6.447238, 0.6]
test= [10.476762, 0.35]
------------------------------
times 5
train= [8.059048, 0.5]
test= [7.2531433, 0.55]
------------------------------
times 6
train= [8.059048, 0.5]
test= [9.670857, 0.4]
------------------------------
times 7
train= [11.282667, 0.3]
test= [9.670857, 0.4]
------------------------------
times 8
train= [9.670857, 0.4]
test= [11.282667, 0.3]
------------------------------
times 9
train= [14.506287, 0.1]
test= [7.2531433, 0.55]
------------------------------
times 10
train= [8.059048, 0.5]
test= [6.447238, 0.6]
------------------------------


In [10]:
# what's wrong?
# all cats but no dogs?(or vise versa?)
# should not see possibilities of 1 (100%) or 0
cnn.predict(test[1])

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [11]:
test[2]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)